In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.2 pyspark-shell"

In [2]:
import findspark
import pandas as pd
findspark.init()
findspark.find()

'/Users/pavel/miniconda3/envs/spark/lib/python3.10/site-packages/pyspark'

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder\
   .master("local")\
   .appName("kafka-example")\
   .getOrCreate()

:: loading settings :: url = jar:file:/Users/pavel/miniconda3/envs/spark/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/pavel/.ivy2/cache
The jars for the packages stored in: /Users/pavel/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-89a9e047-805c-40ac-b5dc-27f4e7565bb2;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central

In [5]:
df = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", 'localhost:29092,localhost:39092') \
        .option('subscribe', 'topic')\
        .load()

In [6]:
from pyspark.sql.types import StringType

In [7]:
df1 = df.withColumn("value", df["value"].cast(StringType()))

In [12]:
df1.show()

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
kafka

In [9]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, BooleanType, LongType, IntegerType

In [13]:
output = df1.writeStream.outputMode("append")\
        .format('parquet')\
        .option("checkpointLocation", './kafka_stream.parquet')\
        .option("path",'./kafka_stream.parquet').start()

23/12/14 15:21:02 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/12/14 15:21:02 WARN StreamingQueryManager: Stopping existing streaming query [id=81e5cdf4-e080-409f-93fd-1d6561e38f26, runId=13e821e3-af4d-41af-b08d-be1fa301997e], as a new run is being started.


23/12/14 15:21:02 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/12/14 15:21:02 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/12/14 15:21:02 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/12/14 15:21:02 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
23/12/14 15:21:02 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


In [16]:
output = df1.writeStream.outputMode("append").format('console').start()

23/12/14 15:24:43 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/bm/6sf1ct396kg9hjg882z2h7_r0000gn/T/temporary-05e434e7-45ba-4006-a65d-b862f4838a8c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/14 15:24:43 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


23/12/14 15:24:43 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/12/14 15:24:43 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/12/14 15:24:43 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/12/14 15:24:43 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
23/12/14 15:24:43 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+



In [17]:
output.awaitTermination()

-------------------------------------------
Batch: 1
-------------------------------------------
+----+--------------------+-----+---------+------+--------------------+-------------+
| key|               value|topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----+---------+------+--------------------+-------------+
|null|{"3": "{\"name\":...|topic|        0|    16|2023-12-14 15:24:...|            0|
+----+--------------------+-----+---------+------+--------------------+-------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+--------------------+-----+---------+------+--------------------+-------------+
| key|               value|topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----+---------+------+--------------------+-------------+
|null|{"3": "{\"name\":...|topic|        0|    17|2023-12-14 15:24:...|            0|
+----+--------------------+-----+---------+------+--------------------+-------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+----+--------------------+-----+---------+------+--------------------+-------------+
| key|               value|topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----+---------+------+--------------------+-------------+
|null|{"3": "{\"name\":...|topic|        0|    18|2023-12-14 15:25:...|            0|
+----+--------------------+----

ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/Users/pavel/miniconda3/envs/spark/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/pavel/miniconda3/envs/spark/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/pavel/miniconda3/envs/spark/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

23/12/14 15:54:23 WARN NetworkClient: [AdminClient clientId=adminclient-3] Connection to node 2 (localhost/127.0.0.1:39092) could not be established. Broker may not be available.
23/12/14 15:54:23 WARN NetworkClient: [AdminClient clientId=adminclient-2] Connection to node 2 (localhost/127.0.0.1:39092) could not be established. Broker may not be available.
23/12/14 15:54:23 WARN NetworkClient: [AdminClient clientId=adminclient-3] Connection to node 1 (localhost/127.0.0.1:29092) could not be established. Broker may not be available.
23/12/14 15:54:23 WARN NetworkClient: [AdminClient clientId=adminclient-2] Connection to node 1 (localhost/127.0.0.1:29092) could not be established. Broker may not be available.
23/12/14 15:54:24 WARN NetworkClient: [AdminClient clientId=adminclient-2] Connection to node 1 (localhost/127.0.0.1:29092) could not be established. Broker may not be available.
23/12/14 15:54:24 WARN NetworkClient: [AdminClient clientId=adminclient-3] Connection to node 2 (localhos

In [14]:
output

AttributeError: 'StreamingQuery' object has no attribute 'show'